In [ ]:
import numpy as np
import math
from scipy.special import gammaincc  
def random_excursions(n, epsilon):
    """
    Random Excursions Test for randomness in a binary sequence.

    Parameters:
    n        : Sequence length
    epsilon  : Binary sequence (list of 0s and 1s)

    Returns:
    p_values : List of p-values for each state x (-4 to 4, excluding 0)
    """
    stateX = [-4, -3, -2, -1, 1, 2, 3, 4]
    counter = [0] * len(stateX)

    pi = np.array([
        [0.0000000000, 0.00000000000, 0.00000000000, 0.00000000000, 0.00000000000, 0.0000000000],
        [0.5000000000, 0.25000000000, 0.12500000000, 0.06250000000, 0.03125000000, 0.0312500000],
        [0.7500000000, 0.06250000000, 0.04687500000, 0.03515625000, 0.02636718750, 0.0791015625],
        [0.8333333333, 0.02777777778, 0.02314814815, 0.01929012346, 0.01607510288, 0.0803755143],
        [0.8750000000, 0.01562500000, 0.01367187500, 0.01196289063, 0.01046752930, 0.0732727051]
    ])

    S_k = np.zeros(n, dtype=int)
    cycle = []

    S_k[0] = 2 * int(epsilon[0]) - 1
    J = 0  # Number of cycles

    for i in range(1, n):
        S_k[i] = S_k[i - 1] + 2 * int(epsilon[i]) - 1
        if S_k[i] == 0:
            J += 1
            cycle.append(i)

    if S_k[n - 1] != 0:
        J += 1
    cycle.append(n)
    nu = np.zeros((6, len(stateX)))
    cycle_start = 0
    cycle_stop = cycle[0]

    for j in range(J):  
        counter = [0] * len(stateX)
        for i in range(cycle_start, cycle_stop):
            if 1 <= abs(S_k[i]) <= 4:
                b = 4 if S_k[i] < 0 else 3
                counter[S_k[i] + b] += 1
        cycle_start = cycle[j] + 1
        cycle_stop = cycle[j + 1] if j + 1 < J else n

        for i in range(len(stateX)):
            if 0 <= counter[i] <= 4:
                nu[counter[i], i] += 1
            elif counter[i] >= 5:
                nu[5, i] += 1
    p_values = []
    for i in range(len(stateX)):
        x = stateX[i]
        chi_square = sum(
            (nu[k, i] - J * pi[abs(x), k]) ** 2 / (J * pi[abs(x), k])
            for k in range(6)
        )
        p_value = gammaincc(2.5, chi_square / 2.0)
        p_values.append(p_value)

    return p_values
